In [140]:
from datetime import date
import pandas as pd
import numpy as np

In [141]:
re_date = pd.read_excel('Inventory Aging Eg.xlsx', sheet_name='Receipt date')
sh_date = pd.read_excel('Inventory Aging Eg.xlsx', sheet_name='Ship date')

In [142]:
# define a function to show the Receipt date of each item in the warehouse
def rpt_date(value, repetitions,Qty):
    return np.repeat(value, repetitions).iloc[Qty:].tolist()

In [143]:
# calculate total qty in sh_date, group by productID
total_qty = sh_date.groupby('ProductID')['Qty'].sum()

In [144]:
# Calculate the sold quantity for each SKU
sh_summary = pd.DataFrame(columns=['ProductID', 'Total Qty'])
sh_summary['ProductID'] = total_qty.index
sh_summary['Total Qty'] = total_qty.values
print(sh_summary)

                      ProductID  Total Qty
0  BEATS.POWERBEATS.PRO-BLUE-NW        142
1           S8-45MM-MIDNIGHT-K1         10


In [145]:
df_age = pd.DataFrame(columns=['ProductID', 'Qty', 'Date Difference'])

for i in re_date['ProductID'].unique().tolist():
  re_date_filtered = re_date[re_date['ProductID'] == i]
  received_dates = re_date_filtered['Received Date']
  received_quantities = re_date_filtered['QtyReceived']
  qty = sh_summary[sh_summary['ProductID'] == i]['Total Qty'].tolist()[0]
  dates = rpt_date(received_dates.astype(str), received_quantities, qty)
  today = date.today()
  date_diff_list = []
  left_qty = sum(received_quantities)-qty
  for j in dates:
    date_obj = date.fromisoformat(j)
    date_diff = today - date_obj
    date_diff_list.append(date_diff.days)
  df_age.loc[len(df_age)] = [i, left_qty, date_diff_list]
print(df_age)

                      ProductID  Qty  \
0  BEATS.POWERBEATS.PRO-BLUE-NW   58   
1           S8-45MM-MIDNIGHT-K1   42   

                                     Date Difference  
0  [464, 464, 464, 464, 464, 464, 464, 464, 464, ...  
1  [33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 3...  


In [146]:
df_age

,ProductID,Qty,Date Difference
0,BEATS.POWERBEATS.PRO-BLUE-NW,58,"[464, 464, 464, 464, 464, 464, 464, 464, 464, ..."
1,S8-45MM-MIDNIGHT-K1,42,"[33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 3..."


In [147]:
# convert "Date Deifference" column in df_age from list to string

df_age['Date Difference'] = df_age['Date Difference'].apply(lambda x: ','.join(map(str, x)))


In [148]:
df_age

,ProductID,Qty,Date Difference
0,BEATS.POWERBEATS.PRO-BLUE-NW,58,"464,464,464,464,464,464,464,464,464,464,464,46..."
1,S8-45MM-MIDNIGHT-K1,42,"33,33,33,33,33,33,33,33,33,33,33,33,33,33,33,3..."


In [149]:
# add four columns "1-30", "31-60","61-90", ">90" to df_age, fill these four columns with count of values from "Date Difference" column for each productID

import re

def categorize_date_diff(date_diff):
  if date_diff <= 30:
    return "Aged 1-30"
  elif date_diff <= 60:
    return "Aged 31-60"
  elif date_diff <= 90:
    return "Aged 61-90"
  else:
    return "Aged >90"

df_age["Aged 1-30"] = 0
df_age["Aged 31-60"] = 0
df_age["Aged 61-90"] = 0
df_age["Aged >90"] = 0

for i, row in df_age.iterrows():
  date_diffs = [int(x) for x in row["Date Difference"].split(",")]
  for date_diff in date_diffs:
    category = categorize_date_diff(date_diff)
    df_age.at[i, category] += 1

df_age.drop(columns=["Date Difference"], inplace=True)

In [151]:
df_age

,ProductID,Qty,Aged 1-30,Aged 31-60,Aged 61-90,Aged >90
0,BEATS.POWERBEATS.PRO-BLUE-NW,58,0,0,0,58
1,S8-45MM-MIDNIGHT-K1,42,6,36,0,0
